In [0]:
bronze_df = (
    spark.read
    .format("delta")
    .table("bronze.orders")
)

In [0]:
display(bronze_df.limit(10))

In [0]:
from pyspark.sql.functions import col, sum

null_counts = bronze_df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in bronze_df.columns
])

null_counts.display()

In [0]:
silver_df = (
    spark.readStream.table("bronze.orders")
    .filter("order_id IS NOT NULL")
    .dropDuplicates(["order_id"])
    .drop("_rescued_data")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/silver/orders_cleaned")
    .trigger(availableNow=True)
    .table("silver.orders_cleaned")
    )


In [0]:
%sql select * from silver.orders_cleaned limit 10;